# Types and Dispatch in Julia

One of the most important goals of high-level languages is to provide *polymorphism*: the ability for the same code to operate on different kinds of values.

Julia uses a vocabulary of *types* for this purpose. Types play the following roles:

- Describe "what kind of thing is this"
- Describe the representation of a value
- Driving *dispatch*: selecting one of several pieces of code
- Driving *specialization*: code is optimized by assuming values have certain types

## Describing values

In [1]:
typeof(3)

Int64

In [2]:
sizeof(Int64)

8

In [3]:
Int64.size

8

In [4]:
isbits(Int64)

true

In [5]:
Int64.mutable

false

In [6]:
supertype(Int64)

Signed

In [7]:
supertype(Signed)

Integer

In [8]:
supertype(Integer)

Real

In [9]:
supertype(Real)

Number

In [10]:
supertype(Number)

Any

In [11]:
# The subtype operator/relation
Integer <: Real

true

In [12]:
String <: Real

false

In [13]:
Any >: String

true

In [14]:
# The `isa` operator/relation
1 isa Int

true

In [15]:
1 isa String

false

Julia has roughly 5 kinds of types. We just saw two:

1. Data types - describing concrete data objects
2. Abstract types - group those together

There are three more:

1. Union types
2. UnionAll types
3. The empty type

## Union types

Expresses a *set union* of types.

In [16]:
1 isa Union{Int,String}

true

In [17]:
"hi" isa Union{Int,String}

true

## UnionAll types

Expresses an *iterated set union* of types.

In [18]:
[1] isa Vector{Int}

true

In [19]:
[1] isa (Vector{T} where T<:Real)

true

$\bigcup\limits_{T<:Real} \tt{Vector}\{T\}$

In [20]:
Union{Vector{Any},Vector{Real}} <: Vector{T} where T>:Real

true

In [21]:
T where T<:Real

Real

In [22]:
rand(1:10,2,2)

2×2 Array{Int64,2}:
 10  9
  3  3

In [23]:
dump(Array)

UnionAll
  var: TypeVar
    name: Symbol T
    lb: Core.TypeofBottom Union{}
    ub: Any
  body: UnionAll
    var: TypeVar
      name: Symbol N
      lb: Core.TypeofBottom Union{}
      ub: Any
    body: Array{T,N} <: DenseArray{T,N}


In [24]:
Vector

Array{T,1} where T

In [ ]:
# Vector = Array{T,1} where T

In [25]:
Vector{Int} <: Vector

true

In [26]:
Vector <: Array

true

In [27]:
Vector{Int} <: Vector{Any}

false

In [28]:
typeintersect((Array{T} where T<:Real), (Array{T,2} where T>:Int))

Array{T,2} where Int64<:T<:Real

In [ ]:
f(a::AbstractArray{<:Number}) = ....

In [29]:
[[2]] isa (Vector{T} where T<:Vector{S} where S<:Integer)

true

## The empty type

Corresponds to the empty set.

In [30]:
1 isa Union{}

false

In [31]:
Union{} <: Int

true

In [32]:
Union{} <: String

true

In [33]:
Union{} <: Array

true

This represents situations where there can't be any value; e.g. an exception is thrown or the program doesn't terminate.

## Dispatch

In [34]:
f(a, b::Any)              = "fallback"
f(a::Number, b::Number)   = "a and b are both numbers"
f(a::Number, b)           = "a is a number"
f(a, b::Number)           = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"

f (generic function with 5 methods)

In [35]:
methods(f)

# 5 methods for generic function "f":
f(a::Integer, b::Integer) in Main at In[34]:5
f(a::Number, b::Number) in Main at In[34]:2
f(a::Number, b) in Main at In[34]:3
f(a, b::Number) in Main at In[34]:4
f(a, b) in Main at In[34]:1

In [36]:
f(1.5, 2)

"a and b are both numbers"

In [37]:
f(1, "string")

"a is a number"

In [38]:
f(1, 2)

"a and b are both integers"

In [39]:
f(1, 2, 3)

LoadError: [91mMethodError: no method matching f(::Int64, ::Int64, ::Int64)[0m
Closest candidates are:
  f(::Integer, ::Integer) at In[34]:5
  f(::Number, ::Number) at In[34]:2
  f(::Number, ::Any) at In[34]:3
  ...[39m

## Tuples

A tuple is an immutable container of any combination of values.

Often used to represent e.g. ordered pairs, or for returning "multiple" values from functions.

In [40]:
t = (1, "hi", 0.33, pi)

(1, "hi", 0.33, π = 3.1415926535897...)

In [41]:
t[2]

"hi"

In [42]:
# "destructuring"
a, b, c = t

(1, "hi", 0.33, π = 3.1415926535897...)

In [43]:
a

1

In [44]:
b

"hi"

In [45]:
typeof(t)

Tuple{Int64,String,Float64,Irrational{:π}}

In [48]:
@which 2.0 * pi

*(x::Number, y::Number) in Base at promotion.jl:250

In [47]:
big(2.0) * pi

6.283185307179586476925286766559005768394338798750211641949889184615632812572396

Tuple types represent the arguments to a function.

In [49]:
first(methods(f)).sig

Tuple{#f,Integer,Integer}

For every function call, the method that gets called is the most specific one such that the argument tuple type is a subtype of the signature.

## "Diagonal" dispatch

In [50]:
d(x::T, y::T) where {T} = "same type"
d(x, y) = "different types"

d (generic function with 2 methods)

In [51]:
d(1, 1)

"same type"

In [52]:
d(1, 2.0)

"different types"

In [53]:
[ m.sig for m in methods(d) ]

2-element Array{Type,1}:
 Tuple{#d,T,T} where T
 Tuple{#d,Any,Any}    

## Variadic (or varargs) methods

In [54]:
v(x...) = (x, "zero or more")

v (generic function with 1 method)

In [55]:
v(x, xs...) = (xs, "one or more")

v (generic function with 2 methods)

In [56]:
v()

((), "zero or more")

In [57]:
v(1)

((), "one or more")

In [58]:
v(1, 2, 3, 4)

((2, 3, 4), "one or more")

## Variadic tuple types

In [59]:
foo(a::Array, Is::Int...) = 0

foo (generic function with 1 method)

In [60]:
first(methods(foo)).sig

Tuple{#foo,Array,Vararg{Int64,N} where N}

In [61]:
vt = Tuple{Array, Vararg{Int}}

Tuple{Array,Vararg{Int64,N} where N}

In [62]:
isa(([1],1,2,3), vt)

true

In [63]:
isa(([1],1,0.02,3), vt)

false

## Specialization in action

Internally, the compiler generates specializations for particular types.

Example: For a 3-argument function `f`, the compiler might decide to generate a specialization for `Tuple{Int, Any, Int}`, if for some reason the second argument isn't important.

In [64]:
addall(t::Tuple) = +(t...)  # "splat"

addall (generic function with 1 method)

In [65]:
addall((10,2,3))

15

In [66]:
@code_typed addall((1,2))

CodeInfo(:(begin 
        return (Base.add_int)((Core.getfield)(t, 1)::Int64, (Core.getfield)(t, 2)::Int64)::Int64
    end))=>Int64

In [68]:
@code_native addall((1,2,3))

	.text
Filename: In[64]
	pushq	%rbp
	movq	%rsp, %rbp
Source line: 1
	movq	8(%rdi), %rax
	addq	(%rdi), %rax
	addq	16(%rdi), %rax
	popq	%rbp
	retq
	nopw	%cs:(%rax,%rax)


In [69]:
function alltrue(f, itr)
    @inbounds for x in itr
        f(x) || return false
    end
    return true
end

alltrue (generic function with 1 method)

In [76]:
@code_llvm alltrue(iseven, (2,4,7))


define i8 @julia_alltrue_61499([3 x i64]* nocapture readonly dereferenceable(24)) #0 !dbg !5 {
top:
  br label %L3

L3:                                               ; preds = %if, %top
  %"#temp#.0" = phi i64 [ 1, %top ], [ %4, %if ]
  %1 = icmp sgt i64 %"#temp#.0", 3
  br i1 %1, label %L26, label %if

if:                                               ; preds = %L3
  %2 = add nsw i64 %"#temp#.0", -1
  %3 = getelementptr [3 x i64], [3 x i64]* %0, i64 0, i64 %2
  %4 = add nuw nsw i64 %"#temp#.0", 1
  %5 = load i64, i64* %3, align 8
  %6 = and i64 %5, 1
  %7 = icmp eq i64 %6, 0
  br i1 %7, label %L3, label %L26

L26:                                              ; preds = %if, %L3
  %merge = phi i8 [ 1, %L3 ], [ 0, %if ]
  ret i8 %merge
}


In [72]:
@which isinteger(1)

isinteger(x::Integer) in Base at number.jl:14

In [73]:
@code_typed alltrue(isinteger, [1,2,3])

CodeInfo(:(begin 
        $(Expr(:inbounds, true))
        #temp# = 1
        3: 
        unless (Base.not_int)((#temp# === (Base.add_int)((Base.arraylen)(itr)::Int64, 1)::Int64)::Bool)::Bool goto 12
        SSAValue(3) = (Base.arrayref)(itr, #temp#)::Int64
        SSAValue(4) = (Base.add_int)(#temp#, 1)::Int64
        #temp# = SSAValue(4) # line 3:
        goto 10
        10: 
        goto 3
        12: 
        $(Expr(:inbounds, :pop)) # line 5:
        return true
    end))=>Bool

In [77]:
@code_llvm alltrue(isinteger, [1,2,3])


define i8 @julia_alltrue_61525(i8** dereferenceable(40)) #0 !dbg !5 {
top:
  ret i8 1
}


## Dispatch, specialization, and performance

Dynamic dispatch is traditionally considered "slow".

Instead of a `call` instruction, you need to do a table lookup procedure first.

However:
1. If types are known, the call target can be looked up at compile time.
2. The cost of dynamic dispatch is well worth it *if* you're dispatching to an optimized kernel.

## What to specialize on?

We can't specialize on *everything* because it would take too long and generate too much code.

There's no fully general and automatic approach.

We specialize on types. That's a reasonable default. If the default's not good enough, move more information into types!

A classic: specializing on the value of an integer.

In [78]:
function sum1n(::Val{N}) where {N}    # given `struct Val{N} end`
    s = 0
    for i = 1:N
        s += i
    end
    return s
end

sum1n (generic function with 1 method)

In [79]:
sum1n(Val{10}())

55

In [80]:
@code_llvm sum1n(Val{10}())


define i64 @julia_sum1n_61528() #0 !dbg !5 {
top:
  ret i64 55
}


In [81]:
sum1n(n::Integer) = sum1n(Val{n}())

sum1n (generic function with 2 methods)

In [82]:
sum1n(20)

210

In [86]:
sum1n(rand(1:100))  # dynamic dispatch to specialized code

15

# "Stupid Dispatch Tricks"

## Trick 1: processing arguments recursively

The compiler's optimizations can be exploited to move parts of your own computations to compile time (thus saving time at run time). The general idea is to represent more information within types, instead of using values.

Example: drop the first element of a tuple.

In [87]:
tuple_tail1(t) = t[2:end]

tuple_tail1 (generic function with 1 method)

In [88]:
tuple_tail1((1,2,"hi"))

(2, "hi")

In [90]:
@code_warntype tuple_tail1((1,2,"hi"))

Variables:
  #self#::#tuple_tail1
  t::Tuple{Int64,Int64,String}

Body:
  begin 
      return $(Expr(:invoke, MethodInstance for getindex(::Tuple{Int64,Int64,String}, ::UnitRange{Int64}), :(Main.getindex), :(t), :($(Expr(:new, UnitRange{Int64}, 2, :((Base.select_value)((Base.sle_int)(2, 3)::Bool, 3, (Base.sub_int)(2, 1)::Int64)::Int64))))))
  end::Tuple


Not good. Key information is represented as integers, and when the compiler sees an integer it generally assumes it doesn't know its value.

- The compiler counts 1, infinity
- The compiler can match things but cannot do arithmetic or comparisons
- It's very good at knowing the types of function arguments

In [91]:
argtail(a, rest...) = rest
tupletail(t) = argtail(t...)

tupletail (generic function with 1 method)

In [92]:
tupletail((1,2,"hi"))

(2, "hi")

In [93]:
@code_typed tupletail((1,2,"hi"))

CodeInfo(:(begin 
        return (Core.tuple)((Core.getfield)(t, 2)::Int64, (Core.getfield)(t, 3)::String)::Tuple{Int64,String}
    end))=>Tuple{Int64,String}

### Exercise

Write a type-inferable function to...

1. reverse a tuple
1. take every other element of a tuple
2. interleave the elements of two tuples

In [94]:
rev(t) = t[end:-1:1]

rev (generic function with 1 method)

In [96]:
@code_typed rev((1,""))

CodeInfo(:(begin 
        $(Expr(:inbounds, false))
        # meta: location range.jl colon 30
        # meta: location range.jl _colon 32
        # meta: location range.jl Type 145
        # meta: location range.jl Type 93
        SSAValue(0) = $(Expr(:invoke, MethodInstance for steprange_last(::Int64, ::Int64, ::Int64), :(Base.steprange_last), 2, -1, 1))
        # meta: pop location
        # meta: pop location
        # meta: pop location
        # meta: pop location
        $(Expr(:inbounds, :pop))
        $(Expr(:inbounds, false))
        # meta: location tuple.jl getindex 23
        #26 = $(Expr(:new, Base.##26#27{Tuple{Int64,String}}, :(t)))
        SSAValue(2) = $(Expr(:new, Base.Generator{StepRange{Int64,Int64},Base.##26#27{Tuple{Int64,String}}}, :(#26), :($(Expr(:new, StepRange{Int64,Int64}, 2, -1, SSAValue(0))))))
        SSAValue(3) = $(Expr(:invoke, MethodInstance for collect(::Base.Generator{StepRange{Int64,Int64},Base.##26#27{Tuple{Int64,String}}}), :(Base.collect), SSAV

In [97]:
rev(t) = argrev(t...)
argrev(a, rest...) = (argrev(rest...)..., a)
argrev() = ()

argrev (generic function with 2 methods)

In [99]:
@code_typed rev((1,2,""))

CodeInfo(:(begin 
        SSAValue(1) = (Core.getfield)(t, 2)::Int64
        SSAValue(2) = (Core.getfield)(t, 3)::String
        return (Core.tuple)(SSAValue(2), SSAValue(1), (Core.getfield)(t, 1)::Int64)::Tuple{String,Int64,Int64}
    end))=>Tuple{String,Int64,Int64}

In [100]:
argskip(a, b, rest...) = (a, argskip(rest...)...)
argskip(a) = (a,)
argskip() = ()
skip(t) = argskip(t...)

skip (generic function with 1 method)

In [104]:
skip((1,2,))

(1,)

### Real-ish example: computing the shape of an indexing operation

In [109]:
index_shape(a::Array, idxs) = ish(a, 1, idxs...)

ish(a, i,   ::Real...)         = ()
ish(a, i,   ::Colon,  rest...) = (size(a,i), ish(a,i+1,rest...)...)
ish(a, i, iv::Vector, rest...) = (length(iv), ish(a,i+1,rest...)...)
ish(a, i,   ::Real,   rest...) = ish(a,i+1,rest...)

ish (generic function with 4 methods)

In [110]:
index_shape(rand(3,4,5), (1,:,[1,2]))

(4, 2)

In [108]:
index_shape(rand(3,4,5), (:,2,[1,2,1,2,1,2,1,2]))

(3, 8)

In [113]:
:

Colon()

## Trick 2: look up "trait" values and re-dispatch

### Functions of types

In [111]:
widen(::Type{Float32}) = Float64

widen (generic function with 1 method)

In [112]:
widen(Float32)

Float64

In [114]:
# We use this for type promotion
promote_type(Int64, Float64)

Float64

In [115]:
1 + 2.5

3.5

This can be used to compute attributes of types, then dispatch on those values.

In [ ]:
# Sample trait
abstract IteratorSize
immutable SizeUnknown <: IteratorSize end
immutable HasLength <: IteratorSize end
immutable HasShape <: IteratorSize end
immutable IsInfinite <: IteratorSize end

In [116]:
collect(zip(2:4, "abc"))

3-element Array{Tuple{Int64,Char},1}:
 (2, 'a')
 (3, 'b')
 (4, 'c')

Now we can define a method that says which value of the trait a certain type has.

This is like using dispatch as a lookup table to find out properties of a combination of values.

In [ ]:
iteratorsize(::Type{T}) where {T<:AbstractArray} = HasShape()

iteratorsize{I1,I2}(::Type{Zip2{I1,I2}}) = zip_iteratorsize(iteratorsize(I1),iteratorsize(I2))

zip_iteratorsize(a, b) = SizeUnknown()
zip_iteratorsize{T}(isz::T, ::T) = isz
zip_iteratorsize(::HasLength, ::HasShape) = HasLength()
zip_iteratorsize(::HasShape, ::HasLength) = HasLength()

In [ ]:
# `collect` gives you all the elements from an iterator as an array
collec(itr) = _collec(itr, eltype(itr), Base.iteratorsize(itr))

In [ ]:
function _collec(itr, T, ::Base.HasLength)
    a = Array{T,1}(length(itr))
    i = 0
    for x in itr
        a[i+=1] = x
    end
    return a
end

In [ ]:
function _collec(itr, T, ::Base.SizeUnknown)
    a = Array{T,1}(0)
    for x in itr
        push!(a, x)
    end
    return a
end